## Lesson 7A: Decoding Models and Information Theory

Up until now, we have mostly been considering models of how various stimulus features are **encoded** in neural activity.

We'll now consider the other side of the coin: given a pattern of neural activity, what can we tell about what stimulus was presented?


In [ ]:
# load matplotlib inline mode
%matplotlib inline

# import some useful libraries
import numpy as np                # numerical analysis linear algebra
import matplotlib as mpl
import matplotlib.pyplot as plt   # plotting

# set some style options
mpl.rcParams['image.origin'] = 'lower'
mpl.rcParams['image.aspect'] = 'auto'
mpl.rcParams['image.cmap'] = 'jet'

## Encoding vs decoding models

We can think of the sensory stimulus and the response it evokes as being random variables. In each trial, a stimulus is selected from the stimulus distribution, and this results in some spiking pattern which comes from a large (but finite) set of possible responses.

<img src="images/l9_distributions.png" alt="V1 tuning" style="width: 500px;"/>

### Encoding models

Goal is to model how the response depends on which stimulus was presented. More formally, a probabilistic model of the response conditional on the stimulus, $p(r|s)$. 

RF models covered in our last lesson are an example. The response at time $t_i$ is modeled as a weighted sum of the present and past values of the stimulus, plus some random noise, $\varepsilon_i$.

$$
r_i = \mathbf{s}_i \cdot \mathbf{h} + \varepsilon_i
$$

If the errors are normally distributed with variance $\sigma^2$, then the conditional probability is simply

$$
p(r|\mathbf{s}) = \mathrm{N}(\mathbf{s} \cdot \mathbf{h}, \sigma^2)
$$

### Decoding models

In contrast, a decoding model attempts to reconstruct the stimulus based on the response. More formally, it represents $p(\mathbf{s}|r)$, the probability that a stimulus was presented conditional on the response that was observed.

Decoding models are related to encoding models through Bayes' rule:

$$
p(\mathbf{s}|r) = \frac{p(r|\mathbf{s})p(\mathbf{s})}{p(r)}
$$

## Signal detection

Let's consider a simple problem in which we have to detect a signal against a noisy background. We'll start by describing the encoding model and then develop a decoding model.

On a given trial, the stimulus $s$ is either present ($s = 1$) or absent ($s = 0$).

Similarly, in each trial, the neuron that we're monitoring generates a response that we'll summarize by the rate $r$.

Let's assume that the conditional distribution of the responses is Gaussian:

$$
p(r|s) = \mathrm{N}(\mu, \sigma^2)
$$

Furthermore, let's say that the average response $\mu$ is a simple linear function of whether or not the stimulus is present:

$$
\mu = \beta_0 + \beta_1 s
$$

Let's look at the distributions in Python:

In [ ]:
from tools.dists import normal

beta  = [10, 5]
sigma = np.sqrt(5)

r = np.arange(0, 30, 0.1)
pr_noise = normal(mean=beta[0], std=sigma)
pr_signal = normal(mean=beta[0] + beta[1], std=sigma)
plt.plot(r, pr_noise.pdf(r), lw=2, label="p(r|s=0)")
plt.plot(r, pr_signal.pdf(r), lw=2, label="p(r|s=1)")
plt.legend()
plt.xlabel("Rate")
plt.ylabel("Probability");

Now let's use Bayes's Rule to derive the decoding model, $p(s|r)$.

First, note that $p(s=1|r) = 1 - p(s=0|r)$, so let's just consider the probability that the stimulus is present.

Next, we need to calculate the marginal response probability $p(r)$, which is done by integrating the conditional response distributions:

$$p(r) = \sum p(s, r) = \sum p(r|s) p(s) $$

If the probability of the signal being present is 0.5, then this is equal to

$$p(r) = \frac{1}{2} p(r|s=0) + \frac{1}{2} p(r|s=1)$$

In [ ]:
ps = 0.5
pr = (1 - ps) * pr_noise.pdf(r) + ps * pr_signal.pdf(r)
plt.plot(r, pr)
plt.xlabel("Rate")
plt.ylabel("p(r)")

Finally, we'll use Bayes's Rule to calculate $p(s|r)$. 

In [ ]:
# Bayes's Rule
psr = (pr_signal.pdf(r) * ps) / pr
plt.plot(r, psr)
plt.xlabel("Rate")
plt.ylabel("p(s=1|r)");

### Exercise

Write a **function** to calculate $p(s|r)$ for any value of $p(s)$. Plot $p(s|r)$ when $p(s) = 0.1$ and when $p(s) = 0.9$. How does the decoding model shift its output based on this prior knowledge?

Consult the Software Carpentry [module for functions](http://swcarpentry.github.io/python-novice-inflammation/08-func/index.html) if you need a refresher.

## ROC analysis

The foregoing approach is called a **naive Bayesian decoder**. It works well if you happen to know or control the stimulus probabilities, and if you can collect enough data to accurately model $p(r|s)$.

However, in the real world, it may not be possible to know what the prior probability of a stimulus is. In this case, the only strategy is to set a **criterion**, or threshold, for deciding whether the signal is present or not.

In [ ]:
plt.plot(r, pr_noise.pdf(r), lw=2, label="p(r|s=0)")
plt.plot(r, pr_signal.pdf(r), lw=2, label="p(r|s=1)")
plt.vlines(12, 0, 0.2, label="criterion")
plt.legend()
plt.xlabel("Rate")
plt.ylabel("Probability");

If the response is above the criterion, the signal is assumed to be present, and if it's below, it's assumed to be absent.

### Exercise

With $p(s) = 0.5$, what criterion would you choose to minimize the probability of an error? Give your best guess and explain why. We'll come up with a more formal solution later.

This problem is at the heart of **signal detection theory** (SDT). SDT applies not only to neurons but to the receiver at the end of any noisy communication channel. A core concept is the **reciever operating characteristic** (ROC), which describes how the receiver's ability to accurately detect the signal depends on where it sets its criterion.

Because the criterion and the signal are both binary, this leads to a 2-by-2 table of possible outcomes. Let's let $y = 0$ represent a response of "no signal" (i.e. below criterion) and $y = 1$ represent a response of "signal" (above criterion):


|  -  | s = 0 | s = 1 |
|----|------|------|
|y = 0  | correct rejection| miss |
|y = 1  | false alarm | hit |

It should be clear that false alarms and correct rejections are complementary, and so are misses and hits. Thus, we only need two values from the table, the false alarm rate and the hit rate. If we let $\gamma$ be the criterion, then these are:

\begin{align}
p(FA) & = p(r > \gamma|s = 0) \\
p(H) & = p(r > \gamma|s = 1)
\end{align}

As the criterion is changed, the false alarm and hit rate also change. You should be able to see this by looking at the plot of the two conditional response distributions above.

Let's illustrate in Python. An important concept we need to cover first is the **cumulative probability distribution** or **cdf**. This is a function that gives the area under the pdf up to the specified value. Given $p(x)$, the cdf is defined as:

$$
P(y) = p(x \leq y) = \int_{-\infty}^y p(x) dx
$$

### Exercise

Define $p(FA)$ and $p(H)$ in terms of the **cdfs** for $p(r|s)$:

\begin{align}
p(FA) & =  \\
p(H) & = 
\end{align}

The Python implementation of the normal distirbution has a method `cdf()` that will evaluate the **cdf** for you.

This allows us to generate a plot of $p(H)$ versus $p(FA)$ for different values of the criterion. This is called the ROC curve.

In [ ]:
gamma = np.arange(5, 20)
pfa = 1 - pr_noise.cdf(gamma)
phit = 1 - pr_signal.cdf(gamma)
plt.plot(pfa, phit, 'o', label=r"$\sigma^2 = {}$".format(5))
plt.legend(loc='lower right')
plt.xlabel("P(FA)");
plt.ylabel("P(H)");

Note that there is an inherent tradeoff. You can only increase the hit rate by also increasing the false alarm rate.

### Exercise

Go back to the original definitions of $p(r|s)$. What do you think will happen if you increase or decrease $\sigma^2$?

Write a function to calculate the ROC as a function of $\sigma^2$. Plot ROCs (using the same array of $\gamma$ values as above) for $\sigma^2 = 2, 10, 20, 100$.

### Excursion

If we assume that the conditional response distributions are normally distributed with equal variance, then we can summarize the relationship between them using a single number, $d'$ (d-prime). This is defined as the difference between the means divided by the standard deviation:

$$
d' = \frac{\beta_1}{\sigma} = \frac{\mu_s - \mu_n}{\sigma}
$$

Furthermore, because the difference between the two means is itself a normal distribution, we can estimate $d'$ from the false alarm and hit rate at a single criterion value:

$$
d' = z(H) - z(FA)
$$

where $z(H)$ is the z-score of the hit rate and $z(FA)$ is the z-score of the false alarm rate. The z-score is simply the inverse of the normal cdf. We can calculate it in Python using the `ppf` method:

In [ ]:
pfa = 1 - pr_noise.cdf(10)
phit = 1 - pr_signal.cdf(10)

std_normal = normal()
d_prime = std_normal.ppf(phit) - std_normal.ppf(pfa)
print("d' =", d_prime)

### Exercise

1. With $\sigma^2 = 5$, verify that $d'$ is the same for any value of $\gamma$ in the support.
2. Verify that this empirical calculation is correct given the value defined for $\beta_1$ above.
3. What is $d'$ when $\sigma^2$ is increased to 10?

## Information Theory

Signal detection theory is powerful, but it's not trival to apply to **discrimination** problems where there are more than two stimuli.

Another way of summarizing decoding models is through information theory, which builds on the foundational work of Claude Shannon.

We need to understand two key concepts: **entropy** and **information**

### Entropy

The idea of uncertainty is inherent to probability theory. If some event has a probability that's less than 1.0, that means that we're uncertain about whether it will happen.

A probability distribution quantifies our uncertainty about all the values a particular random variable can take on.

One way of summarizing the uncertainty of the whole distribution is by its **entropy**, which is defined as follows:

$$
H(R) = - \sum_{r \in R} p(r) \log_2 p(r)
$$

For continuous distributions, the entropy is defined by an integral over the support of the pdf:

$$
H(R) = - \int_{R} p(r) \log_2 p(r) dr
$$

If, as above, a base-2 logarithm is used, then entropy has units of **bits**. You can interpret the entropy as the number of digital bits needed to represent the possible values of the distribution.

For example, a fair coin has equal probability of being heads or tails, which gives it an entropy of 1 bit:

In [ ]:
from tools.dists import bernoulli
coin = bernoulli(p=0.5)
coin.entropy() / np.log(2)

When the natural log is used, the units are called **nats**. The `scipy` distribution functions return entropy in nats, so we have to convert to bits by dividing by $\log 2$.

### Exercise

In constrast, if we have a coin that always lands heads, then there is no uncertainty about the outcome, and we need 0 bits to store it.

Thus, we can infer that the entropy of the Bernoulli distribution depends on its parameter.

Plot the entropy of the Bernoulli distribution as a function of its parameter $p$ (the probability that the value will be 1).

## Information

Information has a lot of colloquial usages, but in information theory it means a reduction in uncertainty.

**Mutual information** is the reduction in uncertainty about the value of one random variable when you know the value of another.

$$
I(S;R) = \sum_{s \in S, r \in R} p(s, r) \log_2 \frac{p(s,r)}{p(s) p(r)}
$$

To understand this formula, think about the definition of a joint probability distribution. If the two variables are independent, that means that their joint distribution is just the product of the marginal distributions, 
$p(s, r) = p(s) p(r)$.

In this case, the last term in the sum above is

$$
\log_2 \frac{p(s,r)}{p(s) p(r)} = \log_2 \frac{p(s)p(r)}{p(s)p(r)} = \log_2 1 = 0
$$

In contrast, if $S$ and $R$ are not mutually independent, then $p(s,r) \neq p(s)p(r)$ and $I(S; R) > 0$. MI is always non-negative.

### Exercise

For the signal detection problem, with $\beta_0 = 10$, $\beta_1 = 5$, $\sigma^2 = 5$, and $p(s) = 0.5$, calculate the mutual information between the signal $S$ and the response $R$.

Calculating the joint probability distributions can be a little tricky, so it helps to factor them out:

\begin{align}
I(S;R) & = \sum_{s \in S, r \in R} p(s, r) \log_2 \frac{p(s,r)}{p(s) p(r)} \\
 & = \sum_{s \in S, r \in R} p(r|s)p(s) \log_2 \frac{p(r|s)}{p(r)} \\
 & = \sum_{s \in S} p(s) \sum_{r \in R} p(r|s) \log_2 \frac{p(r|s)}{p(r)}
\end{align}


In [ ]:
beta  = [10, 5]
sigma = np.sqrt(5)
r = np.arange(0, 30, 0.1)
pr_noise = normal(mean=beta[0], std=sigma)
pr_signal = normal(mean=beta[0] + beta[1], std=sigma)
ps = 0.5

What happens when you change the discretization of $𝑟$? Why? What would you need to do to remove this dependence?

## Information and entropy

Mutual information can also be calculated as a reduction in entropy:

\begin{align}
I(S;R) & = H(S) - H(S|R) \\
I(S;R) & = H(R) - H(R|S)
\end{align}

The second term in each of the equations above is a **conditional entropy**, which can be calculated much like the standard entropy, but integrated over all the possible values of the conditioned variable:

$$
H(R|S) = - \sum_{s \in S} p(s) \sum_{r \in R} p(r|s) \log_2 p(r|s)
$$

In the neural setting, the conditional entropy represents how much noise there is in the response. In other words, if the stimulus is known, how much does the response vary from trial to trial?

One way of interpreting this relationship is as an *information channel* between the stimulus and the neural response. 

<img src="images/l9_information_channel.png" alt="entropy and mutual information" style="width: 400px;"/>

The amount of information carried by this channel can be thought of as either the total amount of information the neuron *could* represent in its response distribution $H(R)$, reduced by the variability in the response that is independent of the stimulus, $H(R|S)$, or as the total amount of uncertainty about the stimulus $H(S)$ reduced by the remaining uncertainty about the stimulus when the response is known, $H(S|R)$.

### Exercise

Use one of the entropy formulas to calculate mutual information for the SDT problem, and verify that it agrees with the value you calculated previously.

## Summary and parting thoughts

Decoding models represent the probability of the stimulus conditional on the response.

They can only represent a **best-case scenario**. They don't actually tell us how information in the neural response is intepreted by neurons downstream.

MI can be used to identify stimulus regimes where a neuron carries more information.

MI can also be used to compare neurons and brain regions.

Calculating MI with small amounts of data requires a lot of assumptions.

## Lesson 7B: Data Structures

This exercise builds on the lesson on Data I/O to introduce some ideas about how to organize your data as you read it into Python.

There are whole courses on data representation, but we will only touch on a few topics of importance.

- using Python **lists** and **dictionaries** and pandas **data frames** to represent hierarchical data
- using the principles of **tidy data** to standardize and simplify tabular data


### Readings

For more information:
 
- Wickham H. (2014) Tidy Data. J Stat Soft [doi:10.18637/jss.v059.i10](https://doi.org/10.18637/jss.v059.i10). This article is very R-centric, but the discussion of tidy data principles is generally applicable.

In [1]:
# setting up the notebook
%matplotlib inline

# import some useful libraries
import pprint
import numpy as np                # numerical analysis linear algebra
import pandas as pd
import matplotlib.pyplot as plt   # plotting

## Review of Python data types

We've encountered a number of different Python data types so far. We've seen **atomic** data types like `int` and `float`, which are just single values, and we've seen **aggregate** data types like `list`, `tuple`, `str`, and `np.array` which can contain more than one value.

What all of the aggregate types we've covered so far have in common is that they are indexed by number. We can access individual elements by numerical index (e.g. `my_array[0]`) and subsets of the sequence by slicing (e.g. `my_array[2:20]`). We saw how arrays can have more than one dimension, which then means we can access rows, columns, and various other kinds of subsets based on a combination of indices and slices (e.g. `col = my_array[:,0]`)

We also saw that some kinds of aggregates support **nesting**. For example, point process data are usually represented by *lists of lists* or *lists of arrays*. Accessing elements of these data structures also requires multiple indices, but with some differences in syntax (e.g. `first_spike = trials[0][0]`).

## Python dictionaries

We're now going to consider the case when the elements in an aggregate are not (necessarily) in a particular order, but where instead they are defined by **labels**. In other words, instead of accessing elements by index (`data[0]`), we want to be able to do something like `data["label"]`.

The standard data type in Python for this kind of aggregate (which is called various things in different languages) is the `dict` (**dictionary**). As with a physical dictionary, each entry in a `dict` has a label (**key**) and a definition (**value**).

There are two ways of creating dictionaries in Python:

In [2]:
# dictionary literal form
d1 = {"a": 1, "b": "something else", 4: "cow"}
d1

{'a': 1, 'b': 'something else', 4: 'cow'}

In [3]:
# functional form can only use string-based keys
dict(a=1, b="something else")

{'a': 1, 'b': 'something else'}

The labels of a `dict` are called **keys**. In Python, keys can be strings, numbers, or anything that's *immutable*. The values in a `dict` can be anything.

You access elements of a `dict` by key using the familiar bracket syntax:

In [4]:
print("a ->", d1["a"])
print("b ->", d1["b"])

a -> 1
b -> something else


Trying to access a non-existent key generates an error:

In [5]:
d1["no"]

KeyError: 'no'

You can add new key/value pairs to a dict by simple assignment:

In [6]:
d1["new"] = 25
print(d1)

{'a': 1, 'b': 'something else', 4: 'cow', 'new': 25}


Keys are unique, so assigning a value to a key that already exists will replace the old value.

In [7]:
d1["new"] = 30
print(d1)

{'a': 1, 'b': 'something else', 4: 'cow', 'new': 30}


You can remove key/value pairs with the `pop` method:

In [8]:
print("4 ->", d1.pop(4))
print(d1)

4 -> cow
{'a': 1, 'b': 'something else', 'new': 30}


## Nested data structures

Because Python `dict` and `list` can contain other `dict` and `list` objects, it's easy to create complex hierarchical data structures.

For example, here's how we might represent some spike time data for a single neuron presented with two different stimuli:

In [9]:
spikes = {'A8': [np.random.uniform(0, 1000, 10), np.random.uniform(0, 1000, 15)],
          'B8': [np.random.uniform(0, 1000, 22), np.random.uniform(0, 1000, 17)]}
cell = {'cell': 'st231_11', 'date': '2019-01-22', 'spikes': spikes}
pprint.pprint(cell)

{'cell': 'st231_11',
 'date': '2019-01-22',
 'spikes': {'A8': [array([756.77061414,  25.11438803, 354.03901533, 404.74818875,
       435.93298668, 140.3106998 , 528.47635319, 514.91753363,
       640.05879115, 661.48214133]),
                   array([434.18812732, 789.717343  ,  34.65705739, 317.71298088,
        75.74946492, 147.01942205, 774.66847328, 311.36061386,
       845.67267056, 220.21546703, 119.5295517 , 342.37293918,
       111.9664202 , 358.5587652 , 843.56880284])],
            'B8': [array([138.21770087, 892.12575774,  58.72015894, 684.76736654,
       479.27499797, 344.72252615, 135.26346185, 209.29168416,
       751.05345235, 355.19252504, 159.86958662, 941.40806408,
       105.74168539, 679.72719442, 475.17484459, 361.16359943,
       773.57835392, 244.98644451, 666.42943871, 343.73919761,
       479.82024135, 192.96999374]),
                   array([886.03313662, 653.83541747, 527.19025106, 999.76644929,
       426.69240381, 334.07558159, 942.88227868,  48.18904481

## Traversing nested data structures

We can access elements of a nested data structure with the bracket operators. Each level needs its own set of brackets.

In [10]:
cell['spikes']['A8'][0]

array([756.77061414,  25.11438803, 354.03901533, 404.74818875,
       435.93298668, 140.3106998 , 528.47635319, 514.91753363,
       640.05879115, 661.48214133])

We can also iterate through the structure using for loops. Iterating through list-type aggregates and dicts is slightly different, though.

In [11]:
# iterating through a list/tuple/array yields the items
# example: print # of spikes in each trial
for trial in cell['spikes']['A8']:
    print(len(trial))

10
15


In [12]:
# when iterating through a dict, we usually want to know the keys and values. Use the `items` method to get both.
for stim, trials in cell['spikes'].items():
    print("%s: %d trials" % (stim, len(trials)))

A8: 2 trials
B8: 2 trials


In [13]:
# if you need to know the index while iterating through a sequence, use the `enumerate` function
for i, trial in enumerate(cell['spikes']['A8']):
    print("trial %d: %d spikes" % (i, len(trial)))

trial 0: 10 spikes
trial 1: 15 spikes


### Nested loops

If you need to traverse at multiple levels, you have to use **nested** loops. Notice how the outer `for` block contains an inner `for` block. The intepreter will loop through the arrays in the trial lists for each stimulus. 

In [14]:
for stim, trials in cell['spikes'].items():
    print("%s:" % stim)
    for i, trial in enumerate(trials):
        print(" trial %d: %d spikes" % (i, len(trial)))

A8:
 trial 0: 10 spikes
 trial 1: 15 spikes
B8:
 trial 0: 22 spikes
 trial 1: 17 spikes


### List or dict?

It's possible to represent the same data in a variety of ways. Take the following examples:

In [15]:
prices     = {'a bear': 100, 'a dog': 20}
itemdict   = {'a bear': {'price': 100}, 'a dog': {'price': 20}}
itemlist   = [{'name': 'a bear', 'price': 100}, {'name': 'a dog', 'price': 20}]

Why choose one form over another? There are a few reasons you might want to go with the second or third option: ordering, clarity, and extensibility.

**Order**: In Python, dicts do not have a guaranteed order. That means when you iterate over a dict with `items()` or any other method, you're not necessarily going to get the same result. If order matters, use a list.

**Clarity**: In `prices`, it's not necessarily clear what the keys or the values in the dict mean, aside from the name of the variable. In `itemdict` and `itemlist`, these meanings are *explicit* rather than *implicit*. Using a more explicit representation can help your data structures be clearer to other users and programmers.

**Extensibility**: What do you do with `prices` if you want to add more information about your items? You really only have the option of creating another object or changing how the data are represented. With `itemdict` and `itemlist`, because each element is a dict, you can easily add new key/value pairs as your understanding of the problem changes. This can be important if you want older code to continue to work on the new data structures. This is also called *backwards compatibility*.

Take-home: think about how you want to represent your data in your program in terms of your current and future needs. It often makes sense to choose more complex representations to preserve flexibility down the line. And the difference in your code may not be that much. Looping over these three structures is practically identical:

In [16]:
for name, price in prices.items():
    print(name, "costs", price)
for name, item in itemdict.items():
    print(name, "costs", item["price"])
for item in itemlist:
    print(item["name"], "costs", item["price"])

a bear costs 100
a dog costs 20
a bear costs 100
a dog costs 20
a bear costs 100
a dog costs 20


## Nested data structures: a real example

Neuroscience experiments almost always have a hierarchical structure. 

Switch to the main tab for your jupyter notebook and look in the `data/starling-example` folder. You should see two subdirectories. These are the names of two units from different animals. The animal name is the first part of the directory name, i.e., `st11` and `st49`. Within each subdirectory, there are 6 files that hold the spike times in response to one of six different stimuli.

How do we load the data? And how should we represent them in Python?

Here's some code to traverse the data. We're now dealing with the filesystem, which has a hierarchical organization. The code demonstrates the use of the function `glob`, which gives us a list of files using **wildcards**, and the `split` function, which divides a string up into parts.

In [17]:
from os import path
import glob

def load_spikes(fname):
    """Load spikes from a file in flat ascii format"""
    with open(fname, "rt") as fp:
        return [np.fromstring(line, sep=" ") for line in fp]

# The '*' character will match any file name, so this `glob` call will return a list
# of all the files in `data/starling-example/`
for dirname in glob.glob("data/starling/spikes/*"):
    neuron = path.basename(dirname)
    print("animal:", neuron.split("_")[0])
    print(" neuron:", neuron)
    for respfile in glob.glob(path.join(dirname, "*")):
        stim = path.splitext(path.basename(respfile))[0]
        print("  stim:", stim)
        for i, trial in enumerate(load_spikes(respfile)):
            print("   trial", i, ":", len(trial), "spikes")
        

animal: st11
 neuron: st11_1_2_1
  stim: A0
   trial 0 : 18 spikes
   trial 1 : 10 spikes
   trial 2 : 9 spikes
   trial 3 : 2 spikes
   trial 4 : 1 spikes
  stim: A8
   trial 0 : 23 spikes
   trial 1 : 28 spikes
   trial 2 : 18 spikes
   trial 3 : 18 spikes
   trial 4 : 16 spikes
  stim: B0
   trial 0 : 1 spikes
   trial 1 : 1 spikes
   trial 2 : 1 spikes
   trial 3 : 3 spikes
   trial 4 : 7 spikes
  stim: B8
   trial 0 : 36 spikes
   trial 1 : 19 spikes
   trial 2 : 23 spikes
   trial 3 : 21 spikes
   trial 4 : 16 spikes
  stim: C0
   trial 0 : 17 spikes
   trial 1 : 19 spikes
   trial 2 : 19 spikes
   trial 3 : 27 spikes
   trial 4 : 19 spikes
  stim: C8
   trial 0 : 13 spikes
   trial 1 : 14 spikes
   trial 2 : 13 spikes
   trial 3 : 8 spikes
   trial 4 : 8 spikes
animal: st49
 neuron: st49_2_4_1
  stim: A0
   trial 0 : 86 spikes
   trial 1 : 109 spikes
   trial 2 : 104 spikes
   trial 3 : 123 spikes
   trial 4 : 113 spikes
   trial 5 : 102 spikes
   trial 6 : 96 spikes
   trial 7 

The data are clearly nested, like so:

- animal
  - neuron
     - stimulus
        - trial

However, in deciding how to store the data we have some choices to make, because not everything needs to have its own level. 

As an example, the following structures hold the same information, but are not totally equivalent:

In [18]:
nested = {"animal_id": {"neuron_id": {"stimulus_id": {"trial_1": [1,2], "trial_2": [3,4]}}}}
flat = [{"animal": "animal_id", "neuron": "neuron_id", "stimulus": "stimulus_id", "trial": 1, "data": [1,2]},
        {"animal": "animal_id", "neuron": "neuron_id", "stimulus": "stimulus_id", "trial": 2, "data": [3,4]}]
pprint.pprint(nested)
pprint.pprint(flat)

{'animal_id': {'neuron_id': {'stimulus_id': {'trial_1': [1, 2],
                                             'trial_2': [3, 4]}}}}
[{'animal': 'animal_id',
  'data': [1, 2],
  'neuron': 'neuron_id',
  'stimulus': 'stimulus_id',
  'trial': 1},
 {'animal': 'animal_id',
  'data': [3, 4],
  'neuron': 'neuron_id',
  'stimulus': 'stimulus_id',
  'trial': 2}]


In the *flattened* form, the levels are indicated by key/value pairs. In the *nested* form, the levels are indicated by hierarchical nesting. There may be many possible combinations of flattening and nesting.

To decide what's best, start by thinking about what the *natural unit of analysis* is. In other words, what always goes together, and what can be set aside as an incidental property (for now)?

Clearly, individual trials don't have much meaning on their own. They're repetitions that allow us to better estimate the distribution of a neuron's responses to a given stimulus. Similarly, individual stimuli represent a sample of the universe of possible stimuli. 

In contrast, if we're interested in analying how different neurons respond to various stimuli, we may not care right now about which animal the neuron came from. It might therefore make the most sense to have `animal` be a property rather than a separate level.

Note that this doesn't prevent you from doing hierarchical models at a later stage; it just means that you're committing to doing the first step of your analysis with neurons as individual units. Also, there's not one right answer to this question.

Let's load the data into a nested dictionary, with `neuron` as the natural unit of analysis:

In [19]:
spike_data = {}
for dirname in glob.glob("data/starling/spikes/*"):
    neuron = path.basename(dirname)
    animal = neuron.split("_")[0]
    stims = []
    for respfile in glob.glob(path.join(dirname, "*")):
        stim = path.splitext(path.basename(respfile))[0]
        trials = load_spikes(respfile)
        stims.append({"stimulus": stim, "response": trials})
    ndata = {"animal": animal, "stimuli": stims}
    spike_data[neuron] = ndata

### Exercise 7B1

Plot the responses of both neurons to all 6 stimuli. The plot should have 2 columns and 6 rows. Use code from previous assignments to generate the rasters.

## Hierarchical data in pandas

Hierarchical data can also be represented in tables. 

Recall that a pandas `Series` is a lot like a dictionary in that the elements are indexed by labels. However, the values in a series are usually scalars rather than arbitrary objects.

In [20]:
ages = pd.Series([391, 442, 183], index=['st11', 'st22', 'st231'])
ages

st11     391
st22     442
st231    183
dtype: int64

In fact, you can create Series objects from dicts:

In [21]:
pd.Series({'st11': 391, 'st22': 442, 'st231': 183})

st11     391
st22     442
st231    183
dtype: int64

A pandas DataFrame is also like a dictionary, but the values are rows. This is equivalent to a dict of dicts, and this is one way you can create a DataFrame (though note that the table may need to be transposed so that the outer nesting level corresponds to rows rather than columns).

In [22]:
pd.DataFrame({'st11': {'age': 391, 'sex': 'M'}, 'st22': {'age': 442, 'sex': 'F'}}).T

,age,sex
st11,391,M
st22,442,F


### Hierarchical indices

Data can be nested in a pandas table through the use of multiple indices. Here's an example of how a Series of spike counts might look for trials nested under stimuli:

In [23]:
counts = pd.Series([10, 22, 2, 5], index=[['A8', 'A8', 'B8', 'B8'], [0, 1, 0, 1]])
counts

A8  0    10
    1    22
B8  0     2
    1     5
dtype: int64

Single-index `Series` and `DataFrame` objects are accessed with a single index; multi-index objects require more than one index:

In [24]:
counts["A8", 0]

10

This is conceptually quite similar to how you access data in nested dicts and lists, with some small differences in syntax:

In [25]:
nested["animal_id"]["neuron_id"]["stimulus_id"]["trial_1"][0]

1

However, because the data is more structured, multi-indexed pandas objects give you the ability to select rows based on any of the indices. For example, this pulls out all the rows where `trial` is 0.

In [26]:
counts[:, 0]

A8    10
B8     2
dtype: int64

There is a LOT more you can do with indices, and it's worth giving some careful study to the section on [Hierarchical Indexing](https://jakevdp.github.io/PythonDataScienceHandbook/03.05-hierarchical-indexing.html) in the Python Data Science Handbook.

## Demo

Let's illustrate how we might traverse our spike data and calculate spike counts. We'll use standard Python types first and then convert to a pandas array:

In [27]:
spike_counts = []
for neuron, ndata in spike_data.items():
    for stimresp in ndata['stimuli']:
        for i, trial in enumerate(stimresp['response']):
            d = {
                "neuron": neuron, 
                "animal": ndata["animal"], 
                "stimulus": stimresp["stimulus"], 
                "trial": i, 
                "count": len(trial)
                }
            spike_counts.append(d)
spike_counts

[{'neuron': 'st11_1_2_1',
  'animal': 'st11',
  'stimulus': 'A0',
  'trial': 0,
  'count': 18},
 {'neuron': 'st11_1_2_1',
  'animal': 'st11',
  'stimulus': 'A0',
  'trial': 1,
  'count': 10},
 {'neuron': 'st11_1_2_1',
  'animal': 'st11',
  'stimulus': 'A0',
  'trial': 2,
  'count': 9},
 {'neuron': 'st11_1_2_1',
  'animal': 'st11',
  'stimulus': 'A0',
  'trial': 3,
  'count': 2},
 {'neuron': 'st11_1_2_1',
  'animal': 'st11',
  'stimulus': 'A0',
  'trial': 4,
  'count': 1},
 {'neuron': 'st11_1_2_1',
  'animal': 'st11',
  'stimulus': 'A8',
  'trial': 0,
  'count': 23},
 {'neuron': 'st11_1_2_1',
  'animal': 'st11',
  'stimulus': 'A8',
  'trial': 1,
  'count': 28},
 {'neuron': 'st11_1_2_1',
  'animal': 'st11',
  'stimulus': 'A8',
  'trial': 2,
  'count': 18},
 {'neuron': 'st11_1_2_1',
  'animal': 'st11',
  'stimulus': 'A8',
  'trial': 3,
  'count': 18},
 {'neuron': 'st11_1_2_1',
  'animal': 'st11',
  'stimulus': 'A8',
  'trial': 4,
  'count': 16},
 {'neuron': 'st11_1_2_1',
  'animal': 'st11

Notice how we're flattening the data structure by storing information about each level in individual records. This makes it easier to convert to a pandas DataFrame:

In [28]:
count_df = pd.DataFrame(spike_counts)
count_df

,neuron,animal,stimulus,trial,count
0,st11_1_2_1,st11,A0,0,18
1,st11_1_2_1,st11,A0,1,10
2,st11_1_2_1,st11,A0,2,9
3,st11_1_2_1,st11,A0,3,2
4,st11_1_2_1,st11,A0,4,1
...,...,...,...,...,...
85,st49_2_4_1,st49,C8,5,94
86,st49_2_4_1,st49,C8,6,87
87,st49_2_4_1,st49,C8,7,79
88,st49_2_4_1,st49,C8,8,101


The final step is to convert some of the columns to indices.

In [29]:
count_df_idx = count_df.set_index(['animal', 'neuron', 'stimulus', 'trial'])
count_df_idx

count
animal neuron     stimulus trial       
st11   st11_1_2_1 A0       0         18
                           1         10
                           2          9
                           3          2
                           4          1
...                                 ...
st49   st49_2_4_1 C8       5         94
                           6         87
                           7         79
                           8        101
                           9         89

[90 rows x 1 columns]

This is what enables us to flexibly select subsets of rows:

In [30]:
count_df_idx.loc["st11",:,"C8"]

count
animal neuron     stimulus trial       
st11   st11_1_2_1 C8       0         13
                           1         14
                           2         13
                           3          8
                           4          8

## Tidy data

What's the best way to organize your tables? Although there may be many good answers, a guiding set of principles that will generally make your life easier falls under the rubric of **tidy data**. As summarized in the Wickham article, tidy data obeys the following principles:

1. Each variable forms a column
2. Each observation forms a row
3. Each type of observational unit forms a table

These principles are fairly easily [applied](https://tomaugspurger.github.io/modern-5-tidy.html) to tables but might require some more creative thinking when dealing with non-tabular data. In general, I try to avoid a lot of nesting, and try to get data into tabular forms as soon as possible.

## Exercise 7B2

Pick one of the CRCNS datasets you and your partner looked at. You'll need to download the data and take a look at how it's organized. Refer to the data description document, and think of a simple visualization you would like to do. For example, you might want to plot a raster of a neuron's response to different stimuli.

The goal of this exercise is to think through how you would structure the data to accomplish this goal.

First, identify the hierarchical structure of the data (or some subset thereof). Common units of analysis include subject, neuron, trial, voxel, electrode. Be sure to distinguish between *properties* of an analysis unit (for example, the sex or age of a subject) and *identifiers* that can be used to group units at different levels.

In the cell below, edit the example to describe this structure:

- subject
    - sex: male or female
    - age: in days
    - neuron
        - recording location: V1, V2, HC, etc.
        - recording date: YYYY-MM-DD
        - stimulus
            - duration: in seconds
            - trial
                - spike times

Next, write some pseudocode to describe an algorithm for traversing your dataset to load the data into your hierarchy. You may want or need to break apart the data into separate problems; for example, the properties of the subjects might already be in a separate table. Discuss with your instructor what the best part of the problem to focus on is.

Pseudocode means that the statements don't have to do anything (for example, you don't need to know the specific function you'll use to read in your data), but you should lay out the loops you'll use and the data structures you'll create.